In [43]:
import cv2
import numpy as np
import random
import os

In [44]:
def add_gaussian_noise(image, mean=0, sigma=25):
    """Adds Gaussian noise to the image."""
    gauss = np.random.normal(mean, sigma, image.shape).astype('uint8')
    noisy = cv2.add(image, gauss)
    return noisy

In [45]:
def add_salt_and_pepper_noise(image, salt_prob=0.02, pepper_prob=0.02):
    """Adds salt-and-pepper noise to the image."""
    noisy = np.copy(image)
    total_pixels = image.size
    salt_pixels = int(salt_prob * total_pixels)
    pepper_pixels = int(pepper_prob * total_pixels)

    # Add salt (white) pixels
    for _ in range(salt_pixels):
        i = random.randint(0, image.shape[0] - 1)
        j = random.randint(0, image.shape[1] - 1)
        noisy[i, j] = 255

    # Add pepper (black) pixels
    for _ in range(pepper_pixels):
        i = random.randint(0, image.shape[0] - 1)
        j = random.randint(0, image.shape[1] - 1)
        noisy[i, j] = 0

    return noisy

In [46]:
def add_speckle_noise(image):
    """Adds speckle noise to the image."""
    speckle = np.random.randn(*image.shape).astype('float32')
    noisy = image + image * speckle
    noisy = np.clip(noisy, 0, 255).astype('uint8')
    return noisy

In [47]:
def rotate_circular_region(image, center=None, angle=None, radius=None):
    """
    Rotates a circular region of an image with random parameters if not provided.
    
    Parameters:
        image (ndarray): Input image.
        center (tuple, optional): Center of the circular region. Random if None.
        angle (float, optional): Angle of rotation. Random if None.
        radius (int, optional): Radius of the circular region. Random if None.
    
    Returns:
        ndarray: Image with the rotated circular region.
    """
    height, width = image.shape[:2]

    # Generate random parameters if not provided
    if center is None:
        center = (
            random.randint(75, width - 75),  # Random x-coordinate
            random.randint(75, height - 75)  # Random y-coordinate
        )
    if angle is None:
        angle = random.choice([30, 45, 60])  # Random angle
    if radius is None:
        radius = random.randint(50, 75)  # Random radius

    # Create a mask for the circular region
    mask = np.zeros_like(image, dtype=np.uint8)
    cv2.circle(mask, center, radius, (255, 255, 255), -1)

    # Extract the circular region using the mask
    circular_roi = cv2.bitwise_and(image, image, mask=mask)

    # Create a bounding box around the circle
    x, y = center
    x1, y1 = x - radius, y - radius
    x2, y2 = x + radius, y + radius
    circular_roi = circular_roi[y1:y2, x1:x2]

    # Ensure the ROI is square (width and height of the circle bounding box)
    if circular_roi.shape[0] != circular_roi.shape[1]:
        size = max(circular_roi.shape[:2])
        circular_roi = cv2.resize(circular_roi, (size, size))

    # Rotate the circular ROI
    matrix = cv2.getRotationMatrix2D((radius, radius), angle, 1.0)
    rotated_roi = cv2.warpAffine(circular_roi, matrix, (2 * radius, 2 * radius))

    # Create a mask for the rotated circle
    rotated_mask = np.zeros((2 * radius, 2 * radius), dtype=np.uint8)
    cv2.circle(rotated_mask, (radius, radius), radius, 255, -1)

    # Extract the rotated circle using the rotated mask
    rotated_circle = cv2.bitwise_and(rotated_roi, rotated_roi, mask=rotated_mask)

    # Place the rotated circle back into the original image
    result = image.copy()
    for i in range(-radius, radius):
        for j in range(-radius, radius):
            if rotated_mask[radius + j, radius + i]:
                xi, yi = x + i, y + j
                if 0 <= xi < result.shape[1] and 0 <= yi < result.shape[0]:
                    result[yi, xi] = rotated_circle[radius + j, radius + i]

    return result

In [48]:
def add_white_smudge(image, num_smudges=None, max_radius=50):
    """
    Adds random white smudge-like blur spots to the fingerprint image.

    Parameters:
        image: Input image (numpy array).
        num_smudges: Number of white smudges to add.
        max_radius: Maximum radius of the smudge region.

    Returns:
        Image with random white smudges.
    """
    # Generate random parameters if not provided
    if num_smudges is None:
        num_smudges = random.choice([2, 3, 4])
    # Ensure the image is grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    smudged_image = image.copy()
    height, width = image.shape
    
    for _ in range(num_smudges):
        # Random center and radius
        center_x = random.randint(0, width - 1)
        center_y = random.randint(0, height - 1)
        radius = random.randint(30, max_radius)

        # Create a white "smudge" on a black background
        smudge = np.zeros((height, width), dtype=np.uint8)
        cv2.circle(smudge, (center_x, center_y), radius, 255, -1)

        # Apply Gaussian blur to the smudge
        smudge = cv2.GaussianBlur(smudge, (radius // 2 * 2 + 1, radius // 2 * 2 + 1), 0)

        # Blend the smudge into the image (lighter smudges)
        # Chinh do sang bang cach tang giam he so thu 4
        smudged_image = cv2.addWeighted(smudged_image, 1, smudge, 0.5, 0)
    
    return smudged_image

In [49]:
def display_images(original, noisy, title="Noisy Image"):
    """Displays the original and noisy images side by side."""
    combined = np.hstack((original, noisy))
    cv2.imshow(title, combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [40]:
def main():
    # Load fingerprint image in grayscale
    image_path = 'C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\test\\s2.png'
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print("Error: Image not found!")
        return

    # Add different types of noise
    gaussian_noisy = add_gaussian_noise(image)
    salt_pepper_noisy = add_salt_and_pepper_noise(image)
    speckle_noisy = add_speckle_noise(image)
    rotated_image = rotate_circular_region(image, (200, 200), 45, 50)
    """
    Rotates a circular region of the image around a center point.
    Parameters:
        image: Input image (numpy array).
        center: Center of the circle (x, y).
        angle: Rotation angle in degrees.
        radius: Radius of the circular region.
    """

    # Display and save results
    display_images(image, gaussian_noisy, "Gaussian Noise")
    display_images(image, salt_pepper_noisy, "Salt and Pepper Noise")
    display_images(image, speckle_noisy, "Speckle Noise")
    display_images(image, rotated_image, "Regional Rotate")
    # Save noisy images
    #cv2.imwrite("gaussian_noisy.jpg", gaussian_noisy)
    #cv2.imwrite("salt_pepper_noisy.jpg", salt_pepper_noisy)
    #cv2.imwrite("speckle_noisy.jpg", speckle_noisy)
    #cv2.imwrite("rotated_region.jpg", rotated_image)

if __name__ == "__main__":
    main()

In [51]:
def process_images_in_batches(input_folder, output_folder):
    """Processes 4000 images in 800-image batches with different augmentations."""
    os.makedirs(output_folder, exist_ok=True)
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    total_images = len(image_files)

    if total_images < 4000:
        print("Warning: Less than 4000 images found. Processing all available images.")
    else:
        image_files = image_files[:4000]  # Limit to 4000 images

    # Split into 5 portions of 800 images
    batch_size = 800
    augmentation_methods = [
        add_gaussian_noise,           # Gaussian noise
        add_salt_and_pepper_noise,    # Salt and pepper noise
        add_speckle_noise,            # Speckle noise
        lambda img: rotate_circular_region(img),  # Rotating circular region
        "random"  # Random combination
    ]

    for idx, method in enumerate(augmentation_methods):
        start = idx * batch_size
        end = start + batch_size
        current_batch = image_files[start:end]

        print(f"Processing batch {idx + 1} with {method.__name__ if method != 'random' else 'Random Augmentations'}")

        for file in current_batch:
            input_path = os.path.join(input_folder, file)
            image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"Error reading {file}. Skipping.")
                continue

            # Apply augmentation
            if method == "random":
                aug_image = random_augmentation(image)
            else:
                aug_image = method(image)

            # Save the augmented image
            output_path = os.path.join(output_folder, f"batch{idx+1}_{file}")
            cv2.imwrite(output_path, aug_image)

    print("Processing complete.")

def random_augmentation(image):
    """Applies 1-2 random augmentations."""
    methods = [
        #lambda img: add_gaussian_noise(img),
        lambda img: add_salt_and_pepper_noise(img),
        lambda img: add_speckle_noise(img),
        lambda img: rotate_circular_region(img)
    ]
    aug_image = image.copy()
    for _ in range(random.randint(1, 2)):
        method = random.choice(methods)
        aug_image = method(aug_image)
    return aug_image

if __name__ == "__main__":
    input_folder = "C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\model\\anhhvantay"
    output_folder = "C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\test"
    process_images_in_batches(input_folder, output_folder)


Processing batch 1 with add_gaussian_noise
Processing batch 2 with add_salt_and_pepper_noise
Processing batch 3 with add_speckle_noise
Processing batch 4 with <lambda>
Processing batch 5 with Random Augmentations
Processing complete.


In [ ]:
# Load a fingerprint image
image = cv2.imread('C:\\Users\\84965\\Documents\\Fingerprint_Recognition_GROUP8\\model\\anhhvantay\\A_f0006_09.png', cv2.IMREAD_GRAYSCALE)

# Apply rotation with random parameters
augmented_image = add_white_smudge(image)

# Show the result
cv2.imshow("Augmented Image", augmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()